#### *Importando as bibliotecas que serão necessárias*

In [ ]:
import pandas as pd

import numpy as np

from math import log

from Ticker import Ticker

from Tickers import Tickers

from Model import Model

from datetime import datetime

# Importa a biblioteca que será utilizada para criar gráficos interativos.
import plotly.graph_objects as go

# Importa o módulo da biblioteca "typing" que será utilizado para tipar parâmetros opcionais de funções.
from typing import Optional

# Importa o módulo da biblioteca "sklearn" que será utilizado para escalar as features dos DataFrames que serão usados no modelo LSTM.
from sklearn.preprocessing import MinMaxScaler


#### *Obtendo a lista de tickers do S&P500*

In [ ]:
'''
    Essa célula será usada para se obter os tickets das companhias que fazem parte do S&P 500, com base em dados da wikipedia.
'''

# Salva em uma variável a url que contém a tabela com os tickets das companhias.
url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'

# Lê todas as tabelas presentes na url acima.
sp500_table = pd.read_html(url)

# Salva a coluna "Symbol" da primeira tabela em uma variável (tal coluna contém todos os tickets das companhias que fazem parte do S&P 500).
sp500_symbols= sp500_table[0]['Symbol']

# Transforma os tickets obtidos em uma lista.
sp500_symbols = sp500_symbols.tolist()

# Exibe a lista de tickets criada acima.
#sp500_symbols

#### *Definindo alguns parâmetros que serão importantes*

In [ ]:
'''
    Essa célula será usada para criar um dicionário chamado setup, que contém os parâmetros principais que este código necessita. 
'''

# Cria um dicionário para guardar alguns parâmetros importantes.
setup = {
    #
    "symbols": sp500_symbols,
    # Define a data inicial cujos dados serão coletados (Primeiro dia de negociações do S&P500 em 2019).
    "data_extraction_initial_date": datetime(2019,1,2).date(), # Deve obrigatoriamente ser um dia de negociação.
    # Define a data final cujos dados serão coletados (Último dia de negociações do S&P500 em 2023).
    "data_extraction_final_date": datetime(2023,12,30).date(), # Deve obrigatoriamente ser um dia de negociação e deve também obrigatoriamente
                                                               # suceder um dia de negociação.
    # 5 -> Semanal / 20 -> Mensal
    "strategy_time_period": 5,
    "regime_probability_threshold": 0.9,
    "number_of_tickets_to_be_selected": 50,
    "features_time_period": {
        #
        "returns_time_period": 1,
        #
        "exponential_moving_average_time_period": 14,
        #
        "relative_strength_index_time_period": 14,
        #
        "average_true_range_time_period": 14,
        #
        "momemtum_time_period": 14,
        #
        "vix_time_period": "1d",
        # Define o período do cálculo das mudanças percentuais (Usado no vix e no MSGARCH).
        "pct_change_period": 1,
    },
    #
    "lstm_time_sequences_length": 3
}

#### *Obtendo os dados dos tickers*

In [ ]:
#
tickers = Tickers(setup['symbols'], setup['data_extraction_initial_date'], setup['data_extraction_final_date'],
                  setup['features_time_period'], setup['strategy_time_period'],
                  setup['regime_probability_threshold'], setup['number_of_tickets_to_be_selected'])

# Todos os tickers presentes na variável tickers acima estarão padronizados, isto é, possuirão os mesmos índices (esse fato será usado abaixo).

#### *Criando o modelo LSTM*

In [ ]:
#
features_number = (tickers.symbols[0].data.shape[1] - 1)

In [ ]:
model = Model(features_number,setup['lstm_sequence_length'])

In [ ]:
model.create_LSTM_model()

#### **

In [ ]:
def get_ticker_period_results(ticker: Ticker, predicted: pd.Series, period_initial_date: datetime.date, period_final_date: datetime.date):
    
    # Redimensiona y_train para que ele seja bidimensional (Necessário para o MinMaxScaler).
    resized_y_train = ticker.__y_train__.values.reshape(-1, 1)  

    # Cria uma instancia do MinMaxScaler para o target do ticker em questão.
    scaler_target = MinMaxScaler()

    # Ajusta a instância criada acima ao conjunto de treino do target do ticker em questão.
    scaler_target.fit_transform(resized_y_train)

    # Volta os valores de "y_test_scaled_sequences" para a escala normal e associa a esses valores as suas datas originais, criando assim
    # uma série temporal.
    real_y = pd.Series(scaler_target.inverse_transform(ticker.__y_test_scaled_sequences__).flatten(),
                    index= ticker.__y_test__.index[len(ticker.__y_test__) - len(ticker.__y_test_scaled_sequences__):]
    )
    # Adiciona um nome a série temporal "real_y"
    real_y.name = f"Preço real de {ticker.symbol}"

    # Volta os valores estimados para a escala normal e associa a esses valores as suas datas originais, criando assim
    # uma série temporal.
    predicted_y = pd.Series(
        scaler_target.inverse_transform(predicted).flatten(),
        index=ticker.__y_test__.index[len(ticker.__y_test__) - len(ticker.__y_test_scaled_sequences__):]
    )

    results = {
        "Retorno Predito": log(predicted_y[-1]/predicted_y[0]) * 100,
        "Retorno Real": log(real_y[-1]/real_y[0]) * 100
    }

    index = [ticker.symbol + ": " + str(period_initial_date) + " - " + str(period_final_date)]
    
    return pd.DataFrame(results, index=index)

In [ ]:
def process_period(tickers: Tickers, tickers_to_trade:list, period_number:int, strategy_time_period: int, indexes: list, model):
    '''
        Description:

        Args:

        Return:

    '''
    
    #
    period_initial_date = indexes[strategy_time_period*period_number].date()
    #
    period_final_date = indexes[strategy_time_period*period_number - 1 + strategy_time_period].date()
    
    #
    period_results = pd.DataFrame()
    
    #
    for ticker_index in tickers_to_trade: 
        #
        X_train_scaled_sequences, X_test_scaled_sequences, y_train_scaled_sequences, y_test_scaled_sequences = \
        tickers.symbols[ticker_index].prepare_data_for_lstm(period_initial_date,period_final_date,setup['lstm_time_sequences_length'])
        
        #
        predicted, adjusted_error = model.train_model_and_get_results(
            X_train_scaled_sequences,X_test_scaled_sequences,y_train_scaled_sequences,y_test_scaled_sequences)
        
        #
        period_results = pd.concat([period_results,get_ticker_period_results(
            tickers.symbols[ticker_index], predicted, period_initial_date, period_final_date)])
    
    #
    period_results = period_results.sort_values(by="Retorno Predito", ascending=False)
        
    return period_results

In [ ]:
#
periods_number = (tickers.symbols[0].data.shape[0] // setup['strategy_time_period'])

#
indexes = tickers.symbols[0].data.index

#
tickers_to_trade = tickers.get_tickers_to_trade()

In [ ]:
results = []

for period in range(1,5):
    period_results = process_period(tickers, tickers_to_trade[period], period, setup['strategy_time_period'], indexes, model)
    results.append(period_results)

In [ ]:
#
output_file = 'resultados_portfolio_semanal.xlsx'

with pd.ExcelWriter(output_file) as writer:
    for period, period_results in enumerate(results):
        sheet_name = f'Período {period+1}'
        period_results.to_excel(writer, sheet_name=sheet_name)

In [ ]:
for index in range(len(results)):
    print(f"O retorno no período {index+1} foi de aproximadamente {results[index]['Retorno Real'][:20].mean():.2f}%")